# Errors and Exception Handling

In [4]:
import os
import warnings

import bootcamp_utils

In [3]:
!pwd

/Users/hughcomer/git/bootcamp/my_files


In [6]:
os.chdir('/Users/hughcomer/git/bootcamp/')
os.getcwd()

'/Users/hughcomer/git/bootcamp'

Example:

In [8]:
my_str = 'AGCTATC'
my_str[3] = 'G'

TypeError: 'str' object does not support item assignment

In this case, the `TypeError` indicates that we tried to do something that is legal in Python for some types, but we tried to do it to a type for which it is illegal (strings are immutable). In Python, an error detected during execution is called an **exception**. We say that the interpreter **"raised an exception."** There are many kinds of built-in exceptions, and you can find a list of them, with descriptions [here](https://docs.python.org/3/library/exceptions.html#bltin-exceptions). You can write your own kinds of exceptions, but we will not cover that in bootcamp.

In this lesson, we will investigate how to handle errors in your code. Importantly, we will also touch on the different kinds of errors and how to avoid them. Or, more specifically, you will learn how to use exceptions to help you write better, more bug-free code.

## Three types of errors

1. Syntax errors
2. Runtime errors
3. Semantic errors

### Syntax errors

A syntax error in Python would be this:

> my_list = [1, 2, 3

The Python interpreter will raise a SyntaxError exception.

In [9]:
my_list = [1, 2, 3

SyntaxError: unexpected EOF while parsing (<ipython-input-9-34ae6d6cd500>, line 1)

### Runtime errors
Runtime errors occur when a program is syntactically correct, so it can run, but the interpreter encountered something wrong. The example at the start of the tutorial, trying to change a character in a string, is an example of a runtime error. This particular one was a TypeError, which is a more specific type of runtime error. Python does have a `RuntimeError,` which just indicates a generic runtime (non-syntax) error.
Runtime errors are more difficult to spot than syntax errors because it is possible that a program could run all the way through without encountering the error for some inputs, but for other inputs, you get an error. Let's consider the example of a simple function meant to add two numbers.

In [11]:
def add_two_things(a, b):
    """Add two numbers."""
    return a + b

In [12]:
add_two_things('a string', 5.7)

TypeError: must be str, not float

### Semantic errors
Semantic errors are perhaps the most nefarious. They occur when your program is syntactically correct, executes without runtime errors, and then produces the wrong result. These errors are the hardest to find and can do the most damage. After all, when your program does not do what you designed it to do, you want it to scream out with an exception!

Following is a common example of a semantic error in which we change a mutable object within a function and then try to reuse it.

In [13]:
# A function to append a list onto itself, with the intention of 
# returning a new list, but leaving the input unaltered
def double_list(in_list):
    """Append a list to itself."""
    in_list += in_list
    return in_list

# Make a list
my_list = [3, 2, 1]

# Double it
my_list_double = double_list(my_list)

# Later on in our program, we want a sorted my_list
my_list.sort()

# Let's look at my_list:
print('We expect [1, 2, 3]')
print('We get   ', my_list)

We expect [1, 2, 3]
We get    [1, 1, 2, 2, 3, 3]


In [18]:
# A function to append a list onto itself, with the intention of 
# returning a new list, but leaving the input unaltered
def double_list(in_list):
    """Append a list to itself."""
    # make a copy of list using slice of whole list (which makes a copy)
    new_list = in_list[:]
    new_list += in_list
    return new_list

# Make a list
my_list = [3, 2, 1]

# Double it
my_list_double = double_list(my_list)

# Later on in our program, we want a sorted my_list
my_list.sort()

# Let's look at my_list:
print('We expect [1, 2, 3]')
print('We get   ', my_list)
print(my_list_double)

We expect [1, 2, 3]
We get    [1, 2, 3]
[3, 2, 1, 3, 2, 1]


## Handling errors in your code
If you have a syntax error, your code will not even run. So, we will assume we are without syntax errors in this discussion on how to handle errors. So, how can we handle runtime errors? In most use cases, we just write our code and let the Python interpreter tell us about these exceptions. However, sometimes we want to use the fact that we might encounter a runtime error within our code. A common example of this is when importing modules that are convenient, but not essential, for your code to run. Errors are handled in your code using a **try statement.**
Let's try importing a module that computes GC content. This doesn't exist, so we will get an ImportError.

In [19]:
import gc_content

ModuleNotFoundError: No module named 'gc_content'

Now, if we had the gc_content module, we would like to use it. But if not, we will just hand-code a calculation of the GC content of a sequence. We use a try statement.

In [20]:
# Try to get the gc_content module
try:
    import gc_content
    have_gc = True
except ImportError as e:
    have_gc = False
finally:
    # Do whatever is necessary here, like close files
    pass

seq = 'ACGATCTACGATCAGCTGCGCGCATCG'
    
if have_gc:
    print(gc_content(seq))
else:
    print(seq.count('G') + seq.count('C'))

16


The program now runs just fine! The `try` statement consists of an initial try clause. Everything under the try clause is attempted to be executed. If it succeeds, all other clauses of the try statement are skipped, unless there is a `finally` clause, which is always executed. 

If, however, there is an `ImportError`, the code within the `except ImportError as e` clause is executed. The exception does not halt the program. If there is some other kind of error other than an `ImportError`, the interpreter will raise an exception after it does whatever code is in the finally clause. The finally clause is useful to tidy things up, like closing open file handles. While it is possible for a try statement to handle any generic exception by not specifying `ImportError as e`, it is good practice to explicitly specify the exception(s) that you anticipate in try statements as shown here. In this case, we only want to have control over `ImportError`s. We want the interpreter to scream at us for any other, unanticipated errors.

Exception handling is described [here](https://docs.python.org/3/tutorial/errors.html), in the Python tutorial. A better example of `try ... except ... finally` is in the tutorial and copied below:

In [21]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")

divide(2, 1)


divide(2, 0)


divide("2", "1")

result is 2.0
executing finally clause
division by zero!
executing finally clause
executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

## Issuing warnings
We may want to issue a warning instead of silently continuing. For this, the warnings module from the standard library is useful. We use the warnings.warn() method to issue the warning.

In [22]:
# Try to get the gc_content module
try:
    import gc_content
    have_gc = True
except ImportError as e:
    have_gc = False
    warnings.warn('Failed to load gc_content.  Using custom function.', 
                  UserWarning)
finally:
    # Do whatever is necessary here, like close files
    pass

seq = 'ACGATCTACGATCAGCTGCGCGCATCG'
    
if have_gc:
    print(gc_content(seq))
else:
    print(seq.count('G') + seq.count('C'))

16


/Users/hughcomer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Failed to load gc_content.  Using custom function.
  


## Checking input
It is often the case that you want to check the input of a function to ensure that it works properly. In other words, you want to anticipate errors that the user (or you) might make in running your function, and you want to give descriptive error messages. For example, let's say you are writing a code that processes protein sequences that contain only the 20 naturally occurring amino acids represented by their one-letter abbreviation. You may wish to check that the amino acid sequence is legitimate. In particular, the letters `B, J, O, U, X,` and `Z,` are not valid abbreviations for standard amino acids. (We will not use the [ambiguity code](https://en.wikipedia.org/wiki/Amino_acid#Table_of_standard_amino_acid_abbreviations_and_properties), e.g. B for aspartic acid or asparagine, Z for glutamine or glutamic acid, or X for any amino acid.)
To illustrate the point, we will write a simple function that converts the sequence of one-letter amino acids to the three-letter abbreviation. We'll use the dictionary that converts single-letter amino acid codes to triple letter that we encountered in the lesson on dictionaries.

In [23]:
def one_to_three(seq):
    """
    Converts a protein sequence using one-letter abbreviations
    to one using three-letter abbreviations.
    """
    
    # Convert seq to upper case
    seq = seq.upper()
    
    # Do conversion, checking that each input AA is valid
    aa_list = []
    for amino_acid in seq:
        if amino_acid not in bootcamp_utils.aa.keys():
            raise RuntimeError(amino_acid + ' is not a valid amino acid.')
        aa_list += [bootcamp_utils.aa[amino_acid], '-']

    # Return the result
    return ''.join(aa_list[:-1])

So, if we put in a legitimate amino acid sequence, the function works as expected.

In [24]:
one_to_three('waeifnsdfklnsae')

'Trp-Ala-Glu-Ile-Phe-Asn-Ser-Asp-Phe-Lys-Leu-Asn-Ser-Ala-Glu'